# OpenZoo ❤️ LangChain

OpenZoo works out-of-the-box with LangChain!

This is the OpenZoo version of the [Langchain Quickstart](https://python.langchain.com/docs/use_cases/chatbots/quickstart/)

---

### Imports & Setup

In [ ]:
%pip install langchain-openai
%pip install httpx
%pip install sentence-transformers

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

---

### Basic Chat Usage

To instantiate the LLM object using the ChatOpenAI wrapper, just replace 

- 'model_name' with the OpenZoo spec (for example: 'chat')
- 'openai_api_key' with the openzoo api key
- 'openai_api_base' with 'https://api.openzoo.ai/v1'

In [ ]:
from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

OPENZOO_API_KEY = os.getenv("OPENZOO_API_KEY")          # The OpenAI client expects the API key to be set in the OPENAI_API_KEY environment variable
OPENZOO_API_BASE = os.getenv("OPENZOO_API_BASE")        # We reset the API base to use the OpenZoo API

chat = ChatOpenAI(
    model_name="chat M", 
    # openai_api_key="tRuesp85Ip4bbyUnN6R33ONHlGyBhqq6", 
    # openai_api_base="https://api.openzoo.ai/v1"
)

Simple chat

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

---

### Prompt Templates

**Multi-turn chat using prompt templates and memory, using a simple chain**

Create the prompt template and a chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

Inference: the model displays its understanding of the context from memory.

In [ ]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

---

### Message history

LangChain has a MessageHistory class which is useful to manage Chat history

In [ ]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

Add this message history to the previous chat

In [ ]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

Test the ChatBot's memory

In [ ]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

---

### Documents as context

LangChain enables you to use documents as well as the message history to form the context for an inference.

Start by installing a vector store (Chroma) and some requirements

In [ ]:
%pip install --upgrade --quiet langchain-chroma beautifulsoup4
%pip install google-cloud-aiplatform>=1.38.0

Next, we’ll use a document loader to pull data from a webpage:

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

Next, we split it into smaller chunks that the LLM’s context window can handle and store it in a vector database:

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(all_splits, embeddings)

Create a retriever to fetch Documents similar to the query

In [ ]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

Creating a Document chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

Creating a Retrieval Chain

In [ ]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

Test the retrieval

In [ ]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response